In [1]:
import pandas as pd
import statsmodels.formula.api as sm
df = pd.DataFrame.from_csv("StockData.csv", encoding="UTF-8")
model = sm.ols(formula="PXD ~ Index + Oil + Gold + NaturalGas", data=df)

In [2]:
result = model.fit()

In [3]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    PXD   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                     453.1
Date:                Sat, 22 Jul 2017   Prob (F-statistic):          6.19e-267
Time:                        18:39:30   Log-Likelihood:                 4621.8
No. Observations:                1727   AIC:                            -9234.
Df Residuals:                    1722   BIC:                            -9206.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0005      0.000      1.168      0.2

In [4]:
def regress(stock):
    result = sm.ols(formula=stock+" ~ Index + Oil + Gold + NaturalGas", data=df).fit()
    print(result.summary())
regress("F")

                            OLS Regression Results                            
Dep. Variable:                      F   R-squared:                       0.474
Model:                            OLS   Adj. R-squared:                  0.473
Method:                 Least Squares   F-statistic:                     388.7
Date:                Sat, 22 Jul 2017   Prob (F-statistic):          1.13e-238
Time:                        18:39:30   Log-Likelihood:                 5015.0
No. Observations:                1727   AIC:                        -1.002e+04
Df Residuals:                    1722   BIC:                            -9993.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0003      0.000     -0.969      0.3

In [5]:
def regress(stock):
    result = sm.ols(formula=stock+" ~ Index + Oil + Gold + NaturalGas", data=df).fit()
    print(result.pvalues)
regress("AAPL")

Intercept      1.946173e-01
Index         4.033200e-133
Oil            1.547053e-02
Gold           7.595522e-02
NaturalGas     4.721861e-03
dtype: float64


In [6]:
def regress(stock):
    result = sm.ols(formula=stock+" ~ Index + Oil + Gold + NaturalGas", data=df).fit()
    print(result.pvalues<.05)
regress("PXD")

Intercept     False
Index          True
Oil            True
Gold          False
NaturalGas    False
dtype: bool


In [7]:
def regress(stock):
    result = sm.ols(formula=stock+" ~ Index + Oil + Gold + NaturalGas", data=df).fit()
    print(result.pvalues<.05)
    print(result.params)
regress("PXD")

Intercept     False
Index          True
Oil            True
Gold          False
NaturalGas    False
dtype: bool
Intercept     0.000469
Index         1.222012
Oil           0.386156
Gold          0.052714
NaturalGas    0.005414
dtype: float64


In [8]:
def regress(stock):
    result = sm.ols(formula=stock+" ~ Index + Oil + Gold + NaturalGas", data=df).fit()
    truthSeries = result.pvalues<.05
    print(result.params[truthSeries])
regress("PXD")

Index    1.222012
Oil      0.386156
dtype: float64


In [9]:
def regress(stock):
    result = sm.ols(formula=stock+" ~ Index + Oil + Gold + NaturalGas", data=df).fit()
    return pd.DataFrame(result.params[result.pvalues<.05],columns=[stock])
regress("PXD")

,PXD
Index,1.222012
Oil,0.386156


In [10]:
regress("BRK.B")

PatsyError: Error evaluating factor: NameError: name 'BRK' is not defined
    BRK.B ~ Index + Oil + Gold + NaturalGas
    ^^^^^

In [11]:
cols = []
for x in df.columns:
    if "." in x:
        x = x.replace(".","")
    if ":" in x:
        x = x.replace(":","")
    cols.append(x)
df.columns = cols

In [12]:
del df["FTI"]
del df["COL"]
del df['DXC']
del df["BHGE"]

In [13]:
df2 = pd.DataFrame()
for stock in df.columns[4:]:
    frame = regress(stock)
    df2 = pd.concat([df2,frame], axis=1)
print(df2)

                 MMM       ABT      ABBV       ACN      ATVI       AYI  \
Gold             NaN       NaN       NaN       NaN       NaN  0.078922   
Index       0.958352  0.769490  1.098795  0.988583  0.904080  1.317917   
Intercept        NaN       NaN       NaN       NaN       NaN       NaN   
NaturalGas       NaN       NaN       NaN       NaN -0.022059       NaN   
Oil              NaN -0.034686       NaN       NaN -0.046367       NaN   

                ADBE       AMD       AAP       AES    ...         WYNN  \
Gold             NaN       NaN       NaN       NaN    ...     0.150433   
Index       1.171635  1.559588  0.810991  1.121761    ...     1.446935   
Intercept        NaN       NaN       NaN       NaN    ...          NaN   
NaturalGas       NaN       NaN       NaN       NaN    ...          NaN   
Oil              NaN  0.079827 -0.060120  0.081796    ...     0.052993   

                 XEL       XRX      XLNX        XL       XYL       YUM  \
Gold             NaN -0.079305      

In [14]:
df2.to_csv("RegressionMatrix1.csv",encoding="UTF-8")